IMPORTING LIBRARIES

In [4]:
!python --version

Python 3.8.10


In [5]:
# Filtering Warnings
import warnings
warnings.filterwarnings('ignore')

#Import Libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
pd.set_option('display.max_columns', 300) #Setting column display limit
plt.style.use('ggplot') #Applying style to graphs

LOAD DATA

In [6]:
os.makedirs("data", exist_ok=True)
os.makedirs("processed_data", exist_ok=True)
os.makedirs("models", exist_ok=True)

In [7]:

app_df = pd.read_parquet('../data/application_data.parquet')

DATA PRE-PROCESSING

In [8]:

app_df.shape

(307511, 122)

In [9]:

app_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [10]:
app_df.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,104582.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307509.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,134133.000000,3.068510e+05,246546.000000,151450.00000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,151450.000000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,151450.000000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,159080.000000,306490.000000,306490.000000,306490.000000,306490.000000,307510.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.00000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,-4986.120328,-2994.202373,12.061091,0.999997,0.819889,0.199368,0.998133,0.281066,0.056720,2.152665,2.052463,2.031521,12.063419,0.015144,0.050769,0.040659,0.078173,0.230454,0.179555,0.502130,5.143927e-01,0.510853,0.11744,0.088442,0.977735,0.752471,0.044621,0.078942,0.149725,0.226282,0.231894,0.066333,0.100775,0.107399,0.008809,0.028358,0.114231,0.087543,0.977065,0.759637,0.042553,0.074490,0.145193,0.222315,0.228058,0.064958,0.105645,0.105975,0.008076,0.027022,0.117850,0.087955,0.97

NULL CHECKING AND HANDLING

In [11]:
#null checking

(app_df.isnull().sum()/len(app_df)*100).sort_values(ascending = False).head(50)

COMMONAREA_MEDI                 69.872297
COMMONAREA_AVG                  69.872297
COMMONAREA_MODE                 69.872297
NONLIVINGAPARTMENTS_MODE        69.432963
NONLIVINGAPARTMENTS_AVG         69.432963
NONLIVINGAPARTMENTS_MEDI        69.432963
FONDKAPREMONT_MODE              68.386172
LIVINGAPARTMENTS_MODE           68.354953
LIVINGAPARTMENTS_AVG            68.354953
LIVINGAPARTMENTS_MEDI           68.354953
FLOORSMIN_AVG                   67.848630
FLOORSMIN_MODE                  67.848630
FLOORSMIN_MEDI                  67.848630
YEARS_BUILD_MEDI                66.497784
YEARS_BUILD_MODE                66.497784
YEARS_BUILD_AVG                 66.497784
OWN_CAR_AGE                     65.990810
LANDAREA_MEDI                   59.376738
LANDAREA_MODE                   59.376738
LANDAREA_AVG                    59.376738
BASEMENTAREA_MEDI               58.515956
BASEMENTAREA_AVG                58.515956
BASEMENTAREA_MODE               58.515956
EXT_SOURCE_1                    56

It is evident that there are lots of columns with missing values. We need to investigate further and drop columns with null values higher than 40%.

In [ ]:
null_col = app_df.isnull().sum().sort_values(ascending = False)
null_col = null_col[null_col.values >(0.40*len(app_df))]


#Plotting Bar Graph for null values greater than 40%

plt.figure(figsize=(20,4))
null_col.plot(kind='bar', color="#4CB391")
plt.title('List of Columns & null counts where null values are more than 40%')

plt.xlabel("Null Columns",fontdict={"fontsize":12,"fontweight":5})
plt.ylabel("Count of null values",fontdict={"fontsize":12,"fontweight":5})
plt.show()

In [13]:
len(null_col)

49

CORRELATION ANALYSIS

In [14]:
#Correlated columns with target

correlation=app_df.corr(numeric_only=True)
top_positive_corr = correlation.TARGET.sort_values(ascending=False).head(5).index.to_list()
top_negative_corr = correlation.TARGET.sort_values(ascending=True).head(5).index.to_list()
print(top_positive_corr)
print(top_negative_corr)

['TARGET', 'DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'DAYS_LAST_PHONE_CHANGE']
['EXT_SOURCE_3', 'EXT_SOURCE_2', 'EXT_SOURCE_1', 'DAYS_EMPLOYED', 'FLOORSMAX_AVG']


In [15]:
top_corr_columns = []
top_corr_columns = top_positive_corr + top_negative_corr
print(top_corr_columns)

['TARGET', 'DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'DAYS_LAST_PHONE_CHANGE', 'EXT_SOURCE_3', 'EXT_SOURCE_2', 'EXT_SOURCE_1', 'DAYS_EMPLOYED', 'FLOORSMAX_AVG']


In [16]:
cols_to_drop = list(null_col.index.values) #Making list of column names having null values greater than 40%

In [18]:
#Removing top correlated columns from columns to be dropped

for col in top_corr_columns:
    if col in cols_to_drop:
        cols_to_drop.remove(col)
        print(col)

EXT_SOURCE_1
FLOORSMAX_AVG


In [17]:
len(cols_to_drop)

49

In [19]:
app_df.drop(labels = cols_to_drop,axis=1,inplace = True) #Dropping those columns

In [20]:
app_df = app_df[top_corr_columns]

In [21]:
app_df.shape

(307511, 10)

In [22]:
#Checking for left-out columns with null

null_left = (app_df.isnull().sum()/len(app_df)*100).sort_values(ascending = False).head(20)
null_left

EXT_SOURCE_1                   56.381073
FLOORSMAX_AVG                  49.760822
EXT_SOURCE_3                   19.825307
EXT_SOURCE_2                    0.214626
DAYS_LAST_PHONE_CHANGE          0.000325
TARGET                          0.000000
DAYS_BIRTH                      0.000000
REGION_RATING_CLIENT_W_CITY     0.000000
REGION_RATING_CLIENT            0.000000
DAYS_EMPLOYED                   0.000000
dtype: float64

Filling null values

In [23]:
app_df.EXT_SOURCE_3.fillna(app_df.EXT_SOURCE_3.median() , inplace = True)

app_df.EXT_SOURCE_2.fillna(app_df.EXT_SOURCE_2.median() , inplace = True)

app_df.EXT_SOURCE_1.fillna(app_df.EXT_SOURCE_1.median() , inplace = True)

In [25]:
#Since this is an average computation based column, hence we fill NaN values with median

app_df['FLOORSMAX_AVG'].fillna(app_df['FLOORSMAX_AVG'].median(),inplace=True)

In [24]:
#replace few missing values with Mode (the average value)

app_df['DAYS_LAST_PHONE_CHANGE'].fillna(757, inplace=True)

In [26]:
app_df.head()

,TARGET,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,FLOORSMAX_AVG
0,1,-9461,2,2,-1134.0,0.139376,0.262949,0.083037,-637,0.0833
1,0,-16765,1,1,-828.0,0.535276,0.622246,0.311267,-1188,0.2917
2,0,-19046,2,2,-815.0,0.729567,0.555912,0.505998,-225,0.1667
3,0,-19005,2,2,-617.0,0.535276,0.650442,0.505998,-3039,0.1667
4,0,-19932,2,2,-1106.0,0.535276,0.322738,0.505998,-3038,0.1667


In [27]:
#DAYS columns will be standardized by taking absolute values

app_df_days = [i for i in app_df if i.startswith('DAYS')]

app_df[app_df_days] = abs(app_df[app_df_days])
app_df[app_df_days].head()

,DAYS_BIRTH,DAYS_LAST_PHONE_CHANGE,DAYS_EMPLOYED
0,9461,1134.0,637
1,16765,828.0,1188
2,19046,815.0,225
3,19005,617.0,3039
4,19932,1106.0,3038


In [28]:
null_check = (app_df.isnull().sum()/len(app_df)*100).sort_values(ascending = False)
null_check

TARGET                         0.0
DAYS_BIRTH                     0.0
REGION_RATING_CLIENT_W_CITY    0.0
REGION_RATING_CLIENT           0.0
DAYS_LAST_PHONE_CHANGE         0.0
EXT_SOURCE_3                   0.0
EXT_SOURCE_2                   0.0
EXT_SOURCE_1                   0.0
DAYS_EMPLOYED                  0.0
FLOORSMAX_AVG                  0.0
dtype: float64

No further missing values

FEATURE ENGINEERING - BINNING AND GROUPING

In [29]:
app_df.head()

,TARGET,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,FLOORSMAX_AVG
0,1,9461,2,2,1134.0,0.139376,0.262949,0.083037,637,0.0833
1,0,16765,1,1,828.0,0.535276,0.622246,0.311267,1188,0.2917
2,0,19046,2,2,815.0,0.729567,0.555912,0.505998,225,0.1667
3,0,19005,2,2,617.0,0.535276,0.650442,0.505998,3039,0.1667
4,0,19932,2,2,1106.0,0.535276,0.322738,0.505998,3038,0.1667


In [30]:
#DAYS_BIRTH

app_df['DAYS_BIRTH']= (app_df['DAYS_BIRTH']/365).astype(int)
app_df['DAYS_BIRTH'].unique()

array([25, 45, 52, 54, 46, 37, 51, 55, 39, 27, 36, 38, 23, 35, 26, 48, 31,
       50, 40, 30, 68, 43, 28, 41, 32, 33, 47, 57, 65, 44, 64, 21, 59, 49,
       56, 62, 53, 42, 29, 67, 63, 61, 58, 60, 34, 22, 24, 66, 69, 20])

In [31]:
app_df['AGE_GROUP']=pd.cut(app_df['DAYS_BIRTH'],
                         bins=[19,25,40,60,100], labels=['Very_Young','Youth', 'Middle_Age', 'Elder'])

In [32]:
#DAYS_EMPLOYED
#replace 365243 as 0 because it represent unemployed values
app_df['DAYS_EMPLOYED_YEARS'] = (app_df['DAYS_EMPLOYED'].replace(365243, 0))
app_df['DAYS_EMPLOYED_YEARS']= (app_df['DAYS_EMPLOYED_YEARS']/365).astype(int)
app_df['DAYS_EMPLOYED_YEARS'].unique()

array([ 1,  3,  0,  8,  4,  5,  7, 21, 11,  2,  9, 12, 14, 26, 19,  6, 10,
       13, 24, 18, 23, 15, 22, 27, 32, 20, 25, 42, 16, 30, 38, 33, 17, 31,
       28, 29, 36, 34, 40, 39, 35, 37, 41, 44, 43, 48, 46, 45, 47, 49])

In [33]:
bins = [0, 1, 5, 10, 20, 30, 40, 50]
labels = ['unemployed and <1 yr', '1-5 yrs', '5-10 yrs', '10-20 yrs', '20-30 yrs', '30-40 yrs', '40-50 yrs']

app_df['YEARS_EMPLOYED_GROUP'] = pd.cut(app_df['DAYS_EMPLOYED_YEARS'], bins=bins, labels=labels, right=False)

In [34]:
#DAYS_LAST_PHONE_CHANGE

app_df['DAYS_LAST_PHONE_CHANGE_YEARS'] = round(app_df['DAYS_LAST_PHONE_CHANGE'] / 365)
app_df['DAYS_LAST_PHONE_CHANGE_YEARS'].unique()

array([ 3.,  2.,  7.,  4.,  0.,  5.,  6.,  1.,  8.,  9., 11., 10., 12.])

In [35]:
bins = [-1, 1, 4, float('inf')]
labels = ['Recent', 'Moderate', 'Old']
app_df['PHONE_CHANGE_GROUP'] = pd.cut(app_df['DAYS_LAST_PHONE_CHANGE_YEARS'], bins=bins, labels=labels)

In [36]:
app_df.head()

,TARGET,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,FLOORSMAX_AVG,AGE_GROUP,DAYS_EMPLOYED_YEARS,YEARS_EMPLOYED_GROUP,DAYS_LAST_PHONE_CHANGE_YEARS,PHONE_CHANGE_GROUP
0,1,25,2,2,1134.0,0.139376,0.262949,0.083037,637,0.0833,Very_Young,1,1-5 yrs,3.0,Moderate
1,0,45,1,1,828.0,0.535276,0.622246,0.311267,1188,0.2917,Middle_Age,3,1-5 yrs,2.0,Moderate
2,0,52,2,2,815.0,0.729567,0.555912,0.505998,225,0.1667,Middle_Age,0,unemployed and <1 yr,2.0,Moderate
3,0,52,2,2,617.0,0.535276,0.650442,0.505998,3039,0.1667,Middle_Age,8,5-10 yrs,2.0,Moderate
4,0,54,2,2,1106.0,0.535276,0.322738,0.505998,3038,0.1667,Middle_Age,8,5-10 yrs,3.0,Moderate


In [37]:
#Drop these columns to prevent multi-collinearity

app_df.drop(columns=['DAYS_EMPLOYED', 'DAYS_EMPLOYED_YEARS','DAYS_BIRTH','DAYS_LAST_PHONE_CHANGE_YEARS','DAYS_LAST_PHONE_CHANGE'], inplace=True)

In [38]:
app_df.head()

,TARGET,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,FLOORSMAX_AVG,AGE_GROUP,YEARS_EMPLOYED_GROUP,PHONE_CHANGE_GROUP
0,1,2,2,0.139376,0.262949,0.083037,0.0833,Very_Young,1-5 yrs,Moderate
1,0,1,1,0.535276,0.622246,0.311267,0.2917,Middle_Age,1-5 yrs,Moderate
2,0,2,2,0.729567,0.555912,0.505998,0.1667,Middle_Age,unemployed and <1 yr,Moderate
3,0,2,2,0.535276,0.650442,0.505998,0.1667,Middle_Age,5-10 yrs,Moderate
4,0,2,2,0.535276,0.322738,0.505998,0.1667,Middle_Age,5-10 yrs,Moderate


EXPLORATORY DATA ANALYSIS

In [39]:
#DATA IMBALANCE CHECK

target_counts = app_df['TARGET'].value_counts()
target_counts

TARGET
0    282686
1     24825
Name: count, dtype: int64

In [ ]:
# Create a pie chart
plt.figure(figsize=(6, 6))
plt.pie(target_counts, labels=target_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of TARGET Variable')
plt.show()

This reveals that 8.1% of the dataset are defaulting customers while 91.9% are not.

In [41]:
Target0 = app_df.loc[app_df["TARGET"]==0]
Target1 = app_df.loc[app_df["TARGET"]==1]

round(len(Target0)/len(Target1),2)

11.39

The imbalance ratio of non-defaulters to defaulter is "11.39". This simply means for every one defaulter, there is approximately 11 non-defaulters. Hence, ROC AUC curve will be better suited to evaluate selected models in predictive modelling section.

MODELLING

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import pickle

In [43]:
model_df = app_df.copy()

In [44]:
y_label = model_df.pop('TARGET').values
y_label

array([1, 0, 0, ..., 0, 1, 0])

In [45]:
model_df.head()

,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,FLOORSMAX_AVG,AGE_GROUP,YEARS_EMPLOYED_GROUP,PHONE_CHANGE_GROUP
0,2,2,0.139376,0.262949,0.083037,0.0833,Very_Young,1-5 yrs,Moderate
1,1,1,0.535276,0.622246,0.311267,0.2917,Middle_Age,1-5 yrs,Moderate
2,2,2,0.729567,0.555912,0.505998,0.1667,Middle_Age,unemployed and <1 yr,Moderate
3,2,2,0.535276,0.650442,0.505998,0.1667,Middle_Age,5-10 yrs,Moderate
4,2,2,0.535276,0.322738,0.505998,0.1667,Middle_Age,5-10 yrs,Moderate


In [46]:
model_df.shape

(307511, 9)

In [47]:
X_train, X_temp, y_train, y_temp = train_test_split(model_df, y_label, stratify = y_label, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify = y_temp, test_size=0.5, random_state=42)
print('Shape of X_train:',X_train.shape)
print('Shape of X_val:',X_val.shape)
print('Shape of X_test:',X_test.shape)

Shape of X_train: (215257, 9)
Shape of X_val: (46127, 9)
Shape of X_test: (46127, 9)


In [48]:
# Saving the Dataframes into parquet files
X_train.to_parquet('../processed_data/X_train.parquet')
X_val.to_parquet('../processed_data/X_val.parquet')
X_test.to_parquet('../processed_data/X_test.parquet')

# Saving the numpy arrays into text files
np.savetxt('../processed_data/y_train.txt', y_train)
np.savetxt('../processed_data/y_val.txt', y_val)
np.savetxt('../processed_data/y_test.txt', y_test)

STANDARDIZATION

In [49]:
# Seperation of columns into numeric and categorical columns
num_cols = np.array(X_train.select_dtypes(include= ['int64','float64']).columns).tolist()
cat_cols = np.array(X_train.select_dtypes(include= ['category','object']).columns).tolist()

In [50]:
print(cat_cols)
print(num_cols)
print(set(cat_cols + num_cols) - set(X_train.columns))  # Should be empty

['AGE_GROUP', 'YEARS_EMPLOYED_GROUP', 'PHONE_CHANGE_GROUP']
['REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'EXT_SOURCE_3', 'EXT_SOURCE_2', 'EXT_SOURCE_1', 'FLOORSMAX_AVG']
set()


In [51]:
train_dicts = X_train[cat_cols + num_cols].to_dict(orient='records')
val_dicts = X_val[cat_cols + num_cols].to_dict(orient='records')

In [52]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [53]:
#LOGISTIC REGRESSION

lr = LogisticRegression(class_weight='balanced')
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)

# Predict probabilities (for AUC)
y_proba = lr.predict_proba(X_val)[:, 1]  # Probabilities for class 1

# Accuracy
accuracy = round(accuracy_score(y_val, y_pred), 3)

# AUC Score
auc = round(roc_auc_score(y_val, y_proba), 3)

In [54]:
print(f"Pred_Prob: {y_proba}")
print(f"Accuracy: {accuracy}")
print(f"AUC Score: {auc}")

Pred_Prob: [0.43980332 0.26333885 0.33487578 ... 0.64444237 0.5828069  0.42712234]
Accuracy: 0.677
AUC Score: 0.726


In [56]:
# Saving model as a pickle file for the future use
with open('../models/log_reg_credit_pred.bin','wb') as f_out:
    pickle.dump((dv,lr), f_out)

In [55]:
import xgboost as xgb

In [57]:
# XGBOOST

# Convert to DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Set parameters
params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",  # we want early stopping on AUC
    "scale_pos_weight": pd.Series(y_train).value_counts()[0] / pd.Series(y_train).value_counts()[1],  # class imbalance
}

# Watchlist for evaluation
watchlist = [(dtrain, "train"), (dval, "eval")]

# Train with early stopping
booster = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=100,
    evals=watchlist,
    early_stopping_rounds=50,
    verbose_eval=False  # change to True if you want to see training log
)

# Predict
y_proba = booster.predict(dval)  # returns probabilities for class 1
y_pred = (y_proba >= 0.5).astype(int)

# Evaluate
accuracy = round(accuracy_score(y_val, y_pred), 3)
auc = round(roc_auc_score(y_val, y_proba), 3)

print("Accuracy:", accuracy)
print("AUC Score:", auc)

Accuracy: 0.688
AUC Score: 0.724


EXPERIMENT TRACKING

In [59]:
import mlflow

mlflow.set_tracking_uri('sqlite:///cred_sqlite_mlflow.db')
mlflow.set_experiment('credit_default_risk_experiment_tracking')

2025/08/03 16:48:04 INFO mlflow.tracking.fluent: Experiment with name 'credit_default_risk_experiment_tracking' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/mac/Projects/MLops_datatalks_project/02-experiment-tracking/mlruns/1', creation_time=1754236084687, experiment_id='1', last_update_time=1754236084687, lifecycle_stage='active', name='credit_default_risk_experiment_tracking', tags={}>

In [65]:

with mlflow.start_run():
    mlflow.set_tag("engineer", "adeakinwe")
    mlflow.set_tag("model", "Logistic Regression")

    mlflow.log_param("train_data_path", "../processed_data/X_train.parquet")
    mlflow.log_param("val_data_path", "../processed_data/X_val.parquet")

    lr = LogisticRegression(class_weight='balanced')
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    y_proba = lr.predict_proba(X_val)[:, 1]

    accuracy = round(accuracy_score(y_val, y_pred), 3)
    auc = round(roc_auc_score(y_val, y_proba), 3)

    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("auc", auc)

In [66]:
with mlflow.start_run():
    mlflow.set_tag("engineer", "adeakinwe")
    mlflow.set_tag("model", "XGBoost")

    mlflow.log_param("train_data_path", "../processed_data/X_train.parquet")
    mlflow.log_param("val_data_path", "../processed_data/X_val.parquet")

    # Prepare DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)

    # Compute scale_pos_weight if needed
    class_counts = pd.Series(y_train).value_counts()
    scale_pos_weight = class_counts[0] / class_counts[1]

    # XGBoost params
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "scale_pos_weight": scale_pos_weight,
    }

    mlflow.log_param("scale_pos_weight", scale_pos_weight)

    # Train with early stopping
    model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=100,
        evals=[(dtrain, "train"), (dval, "eval")],
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # Predict and evaluate
    y_pred_proba = model.predict(dval)
    y_pred = (y_pred_proba > 0.5).astype(int)

    accuracy = round(accuracy_score(y_val, y_pred), 3)
    auc = round(roc_auc_score(y_val, y_pred_proba), 3)

    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("auc", auc)

hyper-parameter tuning

In [67]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
import mlflow.xgboost

In [ ]:
# Search space for hyperopt
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
}

# Objective function for hyperopt
def objective(params):
    # Calculate imbalance ratio
    neg, pos = np.bincount(y_train)
    scale_pos_weight = neg / pos

    # Add required static params
    params['objective'] = 'binary:logistic'
    params['seed'] = 42
    params['eval_metric'] = 'auc'
    params['scale_pos_weight'] = scale_pos_weight

    run_name = f"xgb-md{params['max_depth']}-lr{params['learning_rate']:.3f}"

    with mlflow.start_run(nested=True, run_name=run_name):
        mlflow.set_tag("model", "XGBoost")
        mlflow.set_tag("engineer", "adeakinwe")

        mlflow.log_param("train_data", "../processed_data/X_train.parquet")
        mlflow.log_param("val_data", "../processed_data/X_val.parquet")

        mlflow.log_params({k: round(v, 5) if isinstance(v, float) else v for k, v in params.items()})

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)

        model = xgb.train(
            params,
            dtrain,
            num_boost_round=200,
            evals=[(dval, 'eval')],
            early_stopping_rounds=50,
            verbose_eval=10
        )

        y_pred_proba = model.predict(dval)
        y_pred = (y_pred_proba > 0.5).astype(int)

        accuracy = round(accuracy_score(y_val, y_pred), 3)
        auc = round(roc_auc_score(y_val, y_pred_proba), 3)

        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("auc", auc)

        mlflow.xgboost.log_model(model, artifact_path="models")

        return {'loss': -auc, 'status': STATUS_OK}

# Run outer MLflow parent run
with mlflow.start_run(run_name="xgboost-hyperopt"):
    trials = Trials()
    best_result = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=50,
        trials=trials
    )

    # Log best hyperparameters found
    mlflow.log_params({f"best_{k}": v for k, v in best_result.items()})

train and save best model

In [ ]:
#best params from hyper parameter tuning
best_params = {
    'max_depth': 4,
    'learning_rate': 0.0566,
    'reg_alpha': 0.04744,
    'reg_lambda': 0.04301,
    'min_child_weight': 1.42993,
    'subsample': 0.65727,
    'colsample_bytree': 0.60704,
    'objective': 'binary:logistic',
    'seed': 42,
    'eval_metric': 'auc',
    'scale_pos_weight': 11.38747
}

with mlflow.start_run(run_name="xgboost-final-auc"):

    mlflow.set_tag("model", "XGBoost")
    mlflow.set_tag("engineer", "adeakinwe")
    mlflow.log_param("train_data_path", "../processed_data/X_train.parquet")
    mlflow.log_param("val_data_path", "../processed_data/X_val.parquet")

    mlflow.log_params(best_params)

    # Prepare DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)

    # Train model
    model = xgb.train(
        best_params,
        dtrain,
        num_boost_round=200,
        evals=[(dval, 'eval')],
        early_stopping_rounds=50,
        verbose_eval=10
    )

    # Evaluate
    y_proba = model.predict(dval)
    auc = round(roc_auc_score(y_val, y_proba), 3)
    mlflow.log_metric("auc", auc)

    # Log model (native)
    mlflow.xgboost.log_model(model, artifact_path="models/xgboost_model")

    # Dump model + vectorizer together
    model_bundle = {
        "model": model,
        "vectorizer": dv  # dict_vectorizer
    }

    bundle_path = "../models/xgb_credit_pred.bin"
    with open(bundle_path, "wb") as f_out:
        pickle.dump(model_bundle, f_out)

    mlflow.log_artifact(bundle_path)

load model with mlflow run id and predict

In [74]:
logged_model = 'runs:/2c2f5792316545ed84ddf88b09b072a9/models/xgboost_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models/xgboost_model
  flavor: mlflow.xgboost
  run_id: 2c2f5792316545ed84ddf88b09b072a9

In [75]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

In [78]:
y_pred = xgboost_model.predict(dval)

In [82]:
y_pred[:10]

array([0.44478536, 0.30572954, 0.3450749 , 0.28052852, 0.7395793 ,
       0.5424973 , 0.23607713, 0.53270024, 0.44794875, 0.2912081 ],
      dtype=float32)